## Qwen3推理

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/anaconda3/envs/qwen3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "./Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [4]:
# prepare the model input
prompt = "介绍一下澳门理工大学"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    # enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    enable_thinking=False
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [6]:
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: 澳门理工大学（Museum of Science and Technology, MSTM）是澳门的一所综合性大学，位于澳门特别行政区。它成立于1992年，是澳门高等教育体系的重要组成部分，也是澳门科技创新和人才培养的重要基地。

澳门理工大学以培养科技人才、推动科研与产业发展为使命，设有多个学科和研究中心，涵盖理工、医学、社会科学等多个领域。其研究方向包括人工智能、生物技术、环境科学、信息技术等，致力于解决全球面临的复杂问题。

澳门理工大学注重国际合作，与多个国家和地区的高校、科研机构建立了合作关系，推动了学术交流与技术合作。它在学术界和科研界具有较高的声誉，是澳门教育和科技发展的核心力量之一。

如果你对澳门理工大学的课程、研究方向或未来发展方向感兴趣，我可以进一步为你解答。


## Tokenizer查看

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./Qwen/Qwen3-0.6B")
text = "介绍一下香港大学"
encoded_with_special = tokenizer.encode(text, add_special_tokens=True)
print("Encoded with special tokens:", encoded_with_special)
decoded_with_special = tokenizer.decode(encoded_with_special, skip_special_tokens=False)
print("Decoded with special tokens:", decoded_with_special)

Encoded with special tokens: [109432, 101926, 99562]
Decoded with special tokens: 介绍一下香港大学


In [8]:
chat = [
    {"role": "user", "content": "What's the weather today in LA?"},
    {"role": "assistant", "content": "The weather in LA is sunny with a high of 75°F."},
    {"role": "user", "content": "Will it rain tomorrow?"},
    {"role": "assistant", "content": "No, it's expected to be clear with a low of 58°F."},
]

encoded_chat = tokenizer.apply_chat_template(chat, tokenize=False, enable_thinking=True)
print("Encoded chat:", encoded_chat)

Encoded chat: <|im_start|>user
What's the weather today in LA?<|im_end|>
<|im_start|>assistant
The weather in LA is sunny with a high of 75°F.<|im_end|>
<|im_start|>user
Will it rain tomorrow?<|im_end|>
<|im_start|>assistant
<think>

</think>

No, it's expected to be clear with a low of 58°F.<|im_end|>



In [9]:
encoded_chat = tokenizer.apply_chat_template(chat, tokenize=False, enable_thinking=False)
print("Encoded chat:", encoded_chat)

Encoded chat: <|im_start|>user
What's the weather today in LA?<|im_end|>
<|im_start|>assistant
The weather in LA is sunny with a high of 75°F.<|im_end|>
<|im_start|>user
Will it rain tomorrow?<|im_end|>
<|im_start|>assistant
<think>

</think>

No, it's expected to be clear with a low of 58°F.<|im_end|>

